In [1]:
!which python

/projappl/project_2013104/pengyan1/venv/bin/python


In [2]:
import sys
import os

folder_path = os.path.abspath("/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages")
if folder_path not in sys.path:
    sys.path.append(folder_path)

sys.path

['/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python310.zip',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/lib-dynload',
 '',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages',
 '/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages']

In [27]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MLP, PointNetConv, fps, global_max_pool, radius
from torchmetrics.functional import jaccard_index
from torchmetrics import JaccardIndex
import torch_geometric.transforms as T
from torch_geometric.datasets import S3DIS
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, knn_interpolate
from torch_geometric.typing import WITH_TORCH_CLUSTER
from torch_geometric.utils import scatter

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [4]:
print(torch.cuda.is_available())

True


In [5]:
test_area = 6  
path = "/scratch/project_2013104/datasets/s3dis/"

# transform and pre_transform
transform = T.Compose([
    T.RandomJitter(0.01),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2)
])
#pre_transform = T.NormalizeScale()"""

In [6]:
train_dataset = S3DIS(root=path, test_area=test_area, train=True, force_reload=True, 
                      transform=transform)#,pre_transform=pre_transform)
test_dataset = S3DIS(root=path, test_area=test_area, train=False, force_reload=True, 
                     transform=transform)#,pre_transform=pre_transform)

Processing...
Done!
Processing...
Done!


In [15]:
batch_size=64
num_workers=10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [16]:
class SAModule(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointNetConv(nn, add_self_loops=False)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch


class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch

In [17]:
class FPModule(torch.nn.Module):
    def __init__(self, k, nn):
        super().__init__()
        self.k = k
        self.nn = nn

    def forward(self, x, pos, batch, x_skip, pos_skip, batch_skip):
        x = knn_interpolate(x, pos, pos_skip, batch, batch_skip, k=self.k)
        if x_skip is not None:
            x = torch.cat([x, x_skip], dim=1)
        x = self.nn(x)
        return x, pos_skip, batch_skip

In [18]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.2, 0.2, MLP([3 + 6, 64, 64, 128])) # 3 (pos) + 6 (x)
        self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

        self.fp3_module = FPModule(1, MLP([1024 + 256, 256, 256]))
        self.fp2_module = FPModule(3, MLP([256 + 128, 256, 128]))
        self.fp1_module = FPModule(3, MLP([128 + 6, 128, 128, 128]))

        self.mlp = MLP([128, 128, 128, num_classes], dropout=0.5, norm=None)

        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 128)
        self.lin3 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)

        fp3_out = self.fp3_module(*sa3_out, *sa2_out)
        fp2_out = self.fp2_module(*fp3_out, *sa1_out)
        x, _, _ = self.fp1_module(*fp2_out, *sa0_out)

        return self.mlp(x).log_softmax(dim=-1)



In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(train_dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0



In [28]:
@torch.no_grad()
def test(loader):
    model.eval()
    jaccard = JaccardIndex(num_classes=loader.dataset.num_classes, task="multiclass").to(device)
    
    for data in loader:
        data = data.to(device)
        outs = model(data)
        preds = outs.argmax(dim=-1)
        jaccard.update(preds, data.y)
    
    return jaccard.compute().item()


In [29]:
import time
for epoch in range(1, 46):
    # Track epoch start time
    start_time = time.perf_counter()
    train()
    iou = test(test_loader)
    # Calculate epoch duration
    epoch_time = time.perf_counter() - start_time
    
    # Print results with time
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}, Time: {epoch_time:.2f}s')

[10/318] Loss: 0.5014 Train Acc: 0.8335
[20/318] Loss: 0.4716 Train Acc: 0.8499
[30/318] Loss: 0.4170 Train Acc: 0.8696
[40/318] Loss: 0.4527 Train Acc: 0.8554
[50/318] Loss: 0.4486 Train Acc: 0.8605
[60/318] Loss: 0.4109 Train Acc: 0.8718
[70/318] Loss: 0.3873 Train Acc: 0.8795
[80/318] Loss: 0.4178 Train Acc: 0.8688
[90/318] Loss: 0.3812 Train Acc: 0.8792
[100/318] Loss: 0.4188 Train Acc: 0.8725
[110/318] Loss: 0.4478 Train Acc: 0.8608
[120/318] Loss: 0.4024 Train Acc: 0.8731
[130/318] Loss: 0.3637 Train Acc: 0.8846
[140/318] Loss: 0.3751 Train Acc: 0.8822
[150/318] Loss: 0.3659 Train Acc: 0.8858
[160/318] Loss: 0.3874 Train Acc: 0.8777
[170/318] Loss: 0.3755 Train Acc: 0.8807
[180/318] Loss: 0.3823 Train Acc: 0.8780
[190/318] Loss: 0.4063 Train Acc: 0.8746
[200/318] Loss: 0.4127 Train Acc: 0.8686
[210/318] Loss: 0.4128 Train Acc: 0.8712
[220/318] Loss: 0.3887 Train Acc: 0.8755
[230/318] Loss: 0.3843 Train Acc: 0.8754
[240/318] Loss: 0.3822 Train Acc: 0.8780
[250/318] Loss: 0.3877 Tr

In [30]:

checkpoint_path = "/scratch/project_2013104/checkpoints/pointnet2_s3dis_transform_seg_x6_45_checkpoint.pth"

# Save model, optimizer state, and any other info needed
torch.save({
    'epoch': 45,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    #'loss': loss,
    #'test_accuracy': test_acc
}, checkpoint_path)

print("Checkpoint saved successfully!")

Checkpoint saved successfully!
